# Neural Network with Cubic Data

## 1. Import Required Libraries

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from collections import OrderedDict #이건 Python collections 임

## 2. Data Generation

In [13]:
num_data = 1000 #데이터 1000개
num_epochs = 5000 #epoch 5000번

x = init.uniform_(torch.Tensor(num_data, 1), -10, 10)
#-10은 균등 분포 최소 값, 10은 균등 분포 최대 값. 균등분포는 모든 요소 확률 같음
y = (x**3) - 3*(x**2) - 9*x -1

noise = init.normal_(torch.FloatTensor(num_data, 1), std=0.5)
#표준편차가 0.5의, num_data x 1 size의 FloatTensor 생성. 정규분포

y_noise = y + noise #실제 데이터로 주어지는 식은 noise가 끼어 있어 제대로 된 방정식을 구해야 한다.

# 3. Model & Optimizer

In [14]:
class Model(nn.Module):
    def __init__(self, input_num=1, output_num=1):
        super(Model, self).__init__()
        
        self.model = nn.Sequential(OrderedDict([
            ("fc1", nn.Linear(input_num, 20)), #Linear 모델 (in_features, out_features, bias=True)
            ("relu1", nn.ReLU()),
            ("fc2", nn.Linear(20, 10)), 
            ("relu2", nn.ReLU()),
            ("fc3", nn.Linear(10, 5)), 
            ("relu3", nn.ReLU()),
            ("fc4", nn.Linear(5, output_num))
        ]))
        
        #Sequential는 컨테이너로 순서대로 추가된다. 여러 개의 레이어를 합쳐 하나의 이름으로 묶을 수 있다.
        #OrderedDict를 사용하면, 개별 레이어와 작업의 이름을 지정해 줄 수 있다. 딕셔너리이므로 key가 중복되어선 안된다.
        #https://pytorch.org/docs/master/nn.html#torch.nn.Sequential
        
        #activate function이 없으면, neural network라도 linear하게 결과를 낼 수 밖에 없다.
        #여기 예에서는 activate function이 없으면, 2차원 함수라도 1차원으로 선을 그어 판단하게 된다.
        #하지만, activate function이 추가되면, 데이터의 분포에 맞춰 곡선으로 그어질 수 있다.
        #Relu 없이 학습해보면 loss 차이를 확인해 볼 수 있다.
        
    def forward(self, x): #x가 Tensor
        return self.model(x)
    
model = Model()

# 4. Train

In [15]:
l_rate = 0.001
criterion = nn.L1Loss() #손실함수
optimizer = optim.SGD(model.parameters(), lr=l_rate) #parameters() 메서드로 해당 객체(모델)의 모든 파라미터를 가져온다.

for i in range(num_epochs):
    optimizer.zero_grad() #optimiser.step으로 업데이트된 그라디언트 값들을 초기화해 줘야 한다.
    outputs = model.forward(x) #학습
    loss = criterion(outputs, y_noise) #예측한 값과, 정답
    loss.backward() #역전파 해 준다.
    optimizer.step() #변수 업데이트
    
    if i % 200 == 0:
        print("epoch {}, \tloss {}".format(i, loss)) 
#원래는 Tensor 값인 x, y를 Variable로 변환해서 traininig 해야 했지만, 4.0 부터는 Tensor 그대로 넣어도 된다.

epoch 0, 	loss 219.978271484375
epoch 200, 	loss 218.924072265625
epoch 400, 	loss 201.21429443359375
epoch 600, 	loss 133.3739776611328
epoch 800, 	loss 125.23332977294922
epoch 1000, 	loss 105.49205780029297
epoch 1200, 	loss 59.75245666503906
epoch 1400, 	loss 47.7353630065918
epoch 1600, 	loss 39.92662048339844
epoch 1800, 	loss 50.334495544433594
epoch 2000, 	loss 28.401138305664062
epoch 2200, 	loss 33.611331939697266
epoch 2400, 	loss 21.707387924194336
epoch 2600, 	loss 41.90111541748047
epoch 2800, 	loss 36.4487190246582
epoch 3000, 	loss 23.176036834716797
epoch 3200, 	loss 32.807003021240234
epoch 3400, 	loss 24.50848960876465
epoch 3600, 	loss 28.645904541015625
epoch 3800, 	loss 18.315731048583984
epoch 4000, 	loss 17.272323608398438
epoch 4200, 	loss 25.647497177124023
epoch 4400, 	loss 15.191964149475098
epoch 4600, 	loss 15.13269329071045
epoch 4800, 	loss 16.005477905273438
